In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import pickle
import seaborn as sns
import os

import vdmlab as vdm

from load_data import get_pos, get_spikes
from tuning_curves_functions import get_tc, get_odd_firing_idx, linearize

import info.R063d3_info as r063d3
import info.R066d4_info as r066d4

In [ ]:
info = r063d3
pos = get_pos(info.pos_mat, info.pxl_to_cm)
speed = vdm.get_speed(pos)

In [ ]:
plt.plot(speed['smoothed'])
plt.axhline(y=0.45)
plt.ylim(0,2)
plt.show()


In [ ]:
decoded = dict()
decoded['position'] = np.array([10., 20., 30., 40.])
decoded['time'] = np.array([0., 1., 2., 3.])

In [ ]:
decoded_sequence = vdm.decode_sequences(decoded, min_length=3, max_jump=9)

In [ ]:
assert np.allclose(len(decoded_sequence['position']), 0)

In [ ]:
len(decoded_sequence['position'])